## ODBO as a pipeline for different datasets
To show the generality of our ODBO method, we combine the notebooks for three different datasets into this one notebook. GB1_2016 has combintorially measured all the possible mutations on the 20^4 space, which is different from the three datasets listed here. We have separate notebooks for GB1_2016.
To use this notebook, please change the global control varibles. 

In [ ]:
import numpy as np
import torch
import pandas as pd
import odbo
import os

### Global control varibles
This section describe the parameters used for three different datasets. We also have a detailed walkthrough for method comparisons using GB1_2016 dataset in the seperated notebooks. Since the experimental setups are different for GB1_2016 and the other datasets, this notebook can still work for GB1_2016, but not ideal. We recommend to use the [GB1_2016_ODBO.ipynb](./GB1_2016_ODBO.ipynb) instead.
We note that we didn't do extensive hyperparameter tuning, so other values can work and might even work better.
Values used in the data collection is listed in the comments. Due to different randomness for different device, the results might differ slightly from the results we obtain using local computer.

In [ ]:
# Dataset must be'GB1_2014', 'Ube4b_2013', 'avGFP_2016'
dataset_name = 'GB1_2014'
# Experiment settings 
random_seed = 9 #Random seed for the trial
search_iter = 100 #Number of new observations, GB1_2014=100, Ube4b_2013=50, avGFP_2016=50
# Initialization method protocol
update_method='correlate'#find round 0 experiments to initiate BO. For the datasets with few changes in the sequences, 'correlate' mode is recommended. 
allow_abundance=False #If we allow the top scoring experiments to take abundance of a mutation in different sites into account.
# Featurization settings
method=['Avg','Max','Avg','Max'] #switching order for feature spaces to overcome local maxima in one certain representation
#GB1_2014 and Ube4b_2013 using ['Avg','Max','Avg','Max'], avGFP_2016 using ['Avg','Avg','Avg','Max']
mode='hybrid' #Feature computing mode. 
# Adaptive search space predicted by XGBOD model (Prescreening step)
threshold_ratio = 0.9 #Use the value of top (1-threshold_ratio) of training measurements as threshold. GB1_2014=0.9, Ube4b=0.95, avGFP_2016=0.8
cMat_plot = False #Plot the confusion matrix to check the accuracy of search space prescreening or not
# BO method settings (Optimization step)
BO_method = 'BO' #Must be 'ODBO_BO' or 'ODBO_TuRBO' or 'BO' or 'TuRBO'
gp_method='gp_regression' #Must be 'gp_regression' or 'robust_regression'
tr_length = [6.4] #Trust region length, used in the TuRBO, GB1_2014=[6.4], Ube4b_2013=[3.2], avGFP_2016=[12.8]. 
batch_size = 1 #Number of new oberservations provided by BO. We found 1 is the most cost-effective experimentally
failure_tolerance =20 #Number of failure iterations to change TR length in TuRBO, GB1_2014=20, Ube4b=10, avGFP_2016=10
data_augmentation = True #If do data augmentation. For double mutations, A1B,C2D==C2D,A1B, i.e.[1,2,0.1,0.2]=[2,1,0.2,0.1], we augment their symetric features to both training and test set
# GB1_2014=True, Ube4b=False, avGFP_2016=False
save_files = True #Save files or not

### Data initalization

In [ ]:
np.random.seed(random_seed)

#For the dataset with very small differences b/w each measurements, we use an exponential scaling to make BO easier
if dataset_name == 'GB1_2014':
    data_test = pd.read_csv('../datasets/GB1_2014_536085.csv', sep=',')
    name_pre, Y_test = np.array(data_test['AACombo']), np.array(data_test['score'])
    Y_test = 2**Y_test 
elif dataset_name == 'Ube4b_2013':
    data_test = pd.read_csv('../datasets/Ube4b_2013_98299.csv', sep=',')
    name_pre, Y_test = np.array(data_test['AACombo']), np.array(data_test['Log2Eratio'])
elif dataset_name == 'avGFP_2016':
    data_test = pd.read_csv('../datasets/avGFP_2016_54025.csv', sep=',')
    name_pre, Y_test = np.array(data_test['AACombo']), np.array(data_test['medianBrightness'])
    Y_test = 2**Y_test
del data_test

#Load the preselected indices using a certain shuffling order. Control Round 0 experiments to be the same for different trials
if os.path.isfile('sele_indices_{}.npy'.format(dataset_name)) == True:
    sele_indices = np.load('sele_indices_{}.npy'.format(dataset_name))
    shuffle_order = np.load('shuffle_order_{}.npy'.format(dataset_name))
    name_pre[1:], Y_test[1:] = name_pre[shuffle_order[1:]], Y_test[shuffle_order[1:]]
    name = odbo.utils.code_to_array(name_pre)    
else:
    # If there is no preselected indices, we initialize by using the settings in the Globall variable
    shuffle_order = np.arange(len(Y_test))
    np.random.shuffle(shuffle_order[1:])
    np.save('shuffle_order_{}.npy'.format(dataset_name), shuffle_order)
    name_pre[1:], Y_test[1:] = name_pre[shuffle_order[1:]], Y_test[shuffle_order[1:]]
    name = odbo.utils.code_to_array(name_pre)
    sele_indices = odbo.initialization.initial_design(name, least_occurance=np.ones(name.shape[1]),allow_abundance=allow_abundance,update_method=update_method,verbose=True)
    np.save('sele_indices_{}.npy'.format(dataset_name), sele_indices)
name_sele, Y_train = name[sele_indices, :], Y_test[sele_indices]
ids_keep = np.delete(range(len(Y_test)), sele_indices)
name, Y_test = name[ids_keep, :], Y_test[ids_keep]
reduction = np.random.choice(np.arange(len(Y_test)), int(0.9*len(Y_test)), replace=False)
name, Y_test = name[reduction, :], Y_test[reduction]
print('Selected initial experiments no. is ', len(Y_train))
print('Select max Y: ', Y_train.max(), 'True max Y:', Y_test.max())


### Featurization and find the adaptive search space model

In [ ]:
# Featurization
feature_model = odbo.featurization.FewFeatureTransform(raw_vars=name_sele, Y=Y_train, method=method[0], mode=mode)
X_test = feature_model.transform(name)
X_train = feature_model.transform(name_sele)

if BO_method == 'ODBO_BO' or BO_method == 'ODBO_TuRBO':
    # Find the threshold for outliers or inliers 
    threshold = Y_train[np.argsort(Y_train)[int(threshold_ratio*len(Y_train))]]
    print('Lowest measurement value to be considered as inlier for BO: ', threshold)
    labels_train = odbo.prescreening.sp_label(X_train, Y_train, thres=threshold)
    # Find the XGBOD adaptive search space model
    pre_model = odbo.prescreening.XGBOD(eval_metric = 'error', random_state = random_seed)
    pre_model.fit(X_train, labels_train)
    # Predict the entire search space to get the adapt search space
    labels_test = odbo.prescreening.sp_label(X_test, Y_test, thres=threshold)
    pred_test_labels = pre_model.predict(X_test)
    sele_id_test = list(np.where(pred_test_labels == 0)[0])
    print("Adapt space size, Entire space size: ", len(sele_id_test), name.shape[0])
    # Plot the confusion matrix to check the accuracy of search space prescreening
    if cMat_plot:
        out_outlier, in_outlier, out_inlier, in_inlier = odbo.plot.plot_cm(labels_test, pred_test_labels, Y_test)
        print("Correct ratio: {0:.3%}".format((len(out_outlier)+len(in_inlier))/len(labels_test)))
        print("FN ratio: {0:.3%}".format(len(out_inlier)/len(labels_test)))
        print("FP ratio: {0:.3%}".format(len(in_outlier)/len(labels_test)))
else:
    sele_id_test = np.arange(len(Y_test))
print("Adapt space size, Entire space size: ", len(sele_id_test), name.shape[0])

### Run optimizations on the datasets

In [ ]:
# Creat search data
X_train_sele, Y_train_sele = torch.tensor(X_train), torch.tensor(Y_train.reshape(len(Y_train),1))
search_name_sele, name_sele_temp = name[sele_id_test, :], name_sele
X_test_sele, Y_test_sele = torch.tensor(X_test[sele_id_test, :]), torch.tensor(Y_test[sele_id_test].reshape(len(sele_id_test),1))

## Run BO experiment with robust regression or directly gp
l, failure_count = 0, 0 #l is the current iteration, failure_count controls when to switch to another featurization
if BO_method == 'ODBO_TuRBO' or BO_method == 'TuRBO':
    state = odbo.turbo.TurboState(dim=X_train_sele.shape[1], batch_size=batch_size, length=tr_length, n_trust_regions=len(tr_length), failure_tolerance = failure_tolerance)
    state.best_value = Y_train_sele.max()

while l < search_iter:
    print("Iter: ", l, "Current Max: ", Y_train_sele.max().detach().numpy(), "Test max: ", Y_test_sele.max().detach().numpy())
    top_ids = np.argsort(Y_train_sele.numpy().ravel())[-40-l:]#To better regress the high meaurement value region, we only pick the top 40+l experiments to regress the surrogate model
    sele_feat = []#There might be some identical features across the current selected training experiments
    for i in range(X_train_sele.shape[1]):
        if (X_train_sele[top_ids,i]-X_train_sele[0,i]).any() !=0:
            sele_feat.append(i)
    X_sele = X_train_sele[:,sele_feat]
    X_test_sele=X_test_sele[:, sele_feat]
    if data_augmentation:
        X_test_sele = torch.cat([X_test_sele, X_test_sele[:, [1,0,3,2]]])
        X_sele = torch.cat([X_sele[top_ids, :], X_sele[top_ids, :][:, [1,0,3,2]]])
        Y_sele = torch.cat([Y_train_sele[top_ids], Y_train_sele[top_ids]])
    else:
        X_sele, Y_sele=X_sele[top_ids, :], Y_train_sele[top_ids]
    # Run optimization using different methods    
    if BO_method == 'ODBO_BO' or BO_method == 'BO':
        X_next, acq_value, next_exp_id = odbo.bo_design(X=X_sele, Y=Y_sele, X_pending=X_test_sele, gp_method=gp_method, batch_size=batch_size)
        next_exp_id = np.mod(next_exp_id, len(Y_test_sele))
    elif BO_method == 'ODBO_TuRBO' or BO_method == 'TuRBO':
        X_next, acq_value, raw_next_exp_id = odbo.turbo_design(state=state, X=X_sele, Y=Y_sele, X_pending=X_test_sele, n_trust_regions=len(tr_length), batch_size=batch_size, gp_method=gp_method)
        Y_next_m = torch.zeros((len(tr_length), batch_size, 1), device=Y_train_sele.device, dtype=Y_train_sele.dtype)
        raw_next_exp_id = np.mod(raw_next_exp_id, len(Y_test_sele))
        next_exp_id = []  
        for i in range(batch_size):
            next_exp_id_m = raw_next_exp_id[:, i]
            Y_next_m[:, i, 0], idtoadd = Y_test_sele[next_exp_id_m].reshape(len(tr_length)), next_exp_id_m[np.argmax(Y_test_sele[next_exp_id_m])]
            next_exp_id.append(idtoadd)

    Y_train_sele = torch.cat([Y_train_sele, Y_test_sele[next_exp_id]])
    ids_keep = list(np.delete(range(Y_test_sele.shape[0]), next_exp_id))
    Y_test_sele = Y_test_sele[ids_keep]
    name_sele_temp = np.concatenate((name_sele_temp, search_name_sele[next_exp_id]))
    search_name_sele = search_name_sele[ids_keep]
    print("Newly added value: ", Y_train_sele[-batch_size:].detach().numpy(), ''.join(name_sele_temp[-1, :]))
    if BO_method == 'ODBO_TuRBO'or BO_method == 'TuRBO':
        # Update the TuRBO state with the newly added Y values
        state = odbo.turbo.update_state(state=state, Y_next=Y_next_m)

    # Switch different representations if one representation fails in 
    if Y_train_sele[-batch_size:].detach().numpy().max() > Y_train_sele[:-batch_size].max():
        failure_count = 0
        feature_model = odbo.featurization.FewFeatureTransform(raw_vars=name_sele_temp, 
                                                                Y=Y_train_sele.detach().numpy(), method=method[1], mode=mode)
    else:
        failure_count = failure_count + 1
        if failure_count >= 3:
            feature_model = odbo.featurization.FewFeatureTransform(raw_vars=name_sele_temp, 
                                                                    Y=Y_train_sele.detach().numpy(), method=method[2], mode=mode)
        else:
            feature_model = odbo.featurization.FewFeatureTransform(raw_vars=name_sele_temp, 
                                                                    Y=Y_train_sele.detach().numpy(), method=method[3], mode=mode)
    del X_test_sele, X_sele, Y_sele
    X_test_sele= torch.tensor(feature_model.transform(search_name_sele))
    X_train_sele = torch.tensor(feature_model.transform(name_sele_temp))
    del feature_model
    l = l + 1

# Save the BO results. Note we save all the observations including the Round 0 ones
if dataset_name == 'GB1_2014' or dataset_name == 'avGFP_2016':
    Y_train_sele = np.log2(Y_train_sele)

if save_files:
    if gp_method == 'robust_regression':
        np.save('results/{}/{}_{}_RobustGP_batch{}_{}.npy'.format(dataset_name, dataset_name, BO_method, batch_size, random_seed), Y_train_sele)
    elif gp_method == 'gp_regression':
        np.save('results/{}/{}_{}_GP_batch{}_{}.npy'.format(dataset_name, dataset_name, BO_method, batch_size, random_seed), Y_train_sele)


### Run random selection on the datasets

In [ ]:
sele_Y, Y_train_sele = list(np.random.choice(Y_test, search_iter, replace = False)), list(Y_train.copy())
Y_train_sele.extend(sele_Y)
print('Max Y', max(sele_Y))
if dataset_name == 'GB1_2014' or dataset_name == 'avGFP_2016':
    Y_train_sele = np.log2(Y_train_sele)
if save_files:
    np.save('results/{}/{}_random_{}.npy'.format(dataset_name, dataset_name, random_seed), Y_train_sele)